In [16]:
import numpy as np
import random
import time
import pygame

class Celula:
    def __init__(self, x, y, cor):
        self._x = x
        self._y = y
        self._cor = cor

    def get_x(self): return self._x
    def set_x(self, x): self._x = x
    def get_y(self): return self._y
    def set_y(self, y): self._y = y
    def get_cor(self): return self._cor
    def set_cor(self, cor): self._cor = cor

In [17]:

class Leucocito(Celula):
    def __init__(self, x, y):
        super().__init__(x, y, (255, 255, 0))  # Amarelo
        self.criacao_tempo = time.time()

    def expirou(self):
        return time.time() - self.criacao_tempo > 7

    def mover(self, max_x, max_y):
        direcoes = [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]
        dx, dy = random.choice(direcoes)
        self.set_x((self.get_x() + dx) % max_x)
        self.set_y((self.get_y() + dy) % max_y)


In [18]:

class Olhos(Celula):
    def __init__(self, x, y):
        super().__init__(x, y, (0, 255, 0))  # Verde

class Nariz(Celula):
    def __init__(self, x, y):
        super().__init__(x, y, (255, 0, 255))  # Roxo

class Boca(Celula):
    def __init__(self, x, y):
        super().__init__(x, y, (255, 0, 0))  # Vermelho


In [19]:

class Virus:
    def __init__(self, x, y, cor):
        self._x = x
        self._y = y
        self._cor = cor

    def mover(self, max_x, max_y):
        direcoes = [(0, -1), (1, 0), (0, 1), (-1, 0)]
        dx, dy = random.choice(direcoes)
        self._x, self._y = (self._x + dx) % max_x, (self._y + dy) % max_y

    def get_x(self): return self._x
    def set_x(self, x): self._x = x
    def get_y(self): return self._y
    def set_y(self, y): self._y = y
    def get_cor(self): return self._cor
    def set_cor(self, cor): self._cor = cor

class Influenza(Virus):
    def __init__(self, x, y):
        super().__init__(x, y, (0, 0, 255))  # Azul
        self.clonado_orgao = False


In [20]:

class Cabeca:
    def __init__(self, largura, altura):
        self.largura = largura
        self.altura = altura
        self.leucocitos = []
        self.virus_influenza = []
        self.olhos = []
        self.nariz = []
        self.boca = []
        self.desenharOrgaos()
        self.popularInicial()

    def desenharOrgaos(self):
        # Órgãos: Olhos, Nariz e Boca.
        for y in range(6, 13):
            for x in range(15, 20):
                self.olhos.append(Olhos(x, y))
            for x in range(40, 45):
                self.olhos.append(Olhos(x, y))
        for y in range(15, 20):
            for x in range(28, 32):
                self.nariz.append(Nariz(x, y))
        for y in range(24, 28):
            for x in range(23, 37):
                self.boca.append(Boca(x, y))
        self.boca.extend([Boca(37, 24), Boca(38, 23), Boca(39, 22), Boca(22, 24), Boca(21, 23), Boca(20, 22)])

    def popularInicial(self):
        for _ in range(10):
            self.add_Leucocito()
        for _ in range(5):
            self.add_Influenza()

    def add_Leucocito(self):
        x, y = self.pos_Aleatoria()
        self.leucocitos.append(Leucocito(x, y))

    def add_Influenza(self):
        x, y = self.pos_Aleatoria()
        self.virus_influenza.append(Influenza(x, y))

    def pos_Aleatoria(self):
        while True:
            x, y = random.randint(0, self.largura - 1), random.randint(0, self.altura - 1)
            if not any(celula.get_x() == x and celula.get_y() == y for celula in self.olhos + self.nariz + self.boca):
                return (x, y)

    def up_Leucocitos(self):
        self.leucocitos = [leucocito for leucocito in self.leucocitos if not leucocito.expirou()]
        while len(self.leucocitos) < 10:
            self.add_Leucocito()

    def moverCelulas(self):
        for leucocito in self.leucocitos:
            leucocito.mover(self.largura, self.altura)
        for virus in self.virus_influenza:
            virus.mover(self.largura, self.altura)

    def intera_Influenza(self):
        for virus in self.virus_influenza[:]:
            if any(celula.get_x() == virus.get_x() and celula.get_y() == virus.get_y() for celula in self.olhos):
                self.add_Influenza()
            elif any(celula.get_x() == virus.get_x() and celula.get_y() == virus.get_y() for celula in self.nariz):
                self.add_Influenza()
            elif any(celula.get_x() == virus.get_x() and celula.get_y() == virus.get_y() for celula in self.boca):
                self.add_Influenza()

    def intera_Leucocito(self):
        for virus in self.virus_influenza[:]:
            if any(leucocito.get_x() == virus.get_x() and leucocito.get_y() == virus.get_y() for leucocito in self.leucocitos):
                self.virus_influenza.remove(virus)
                self.add_Leucocito()

    def desenhar(self, tela):
        tela.fill((255, 255, 255))  # Preenche a tela com branco

        # Desenha as laterais da cabeça
        pygame.draw.rect(tela, (200, 200, 200), (0, 0, self.largura * 10, 10))  # Topo
        pygame.draw.rect(tela, (200, 200, 200), (0, (self.altura - 1) * 10, self.largura * 10, 10))  # Base
        pygame.draw.rect(tela, (200, 200, 200), (0, 0, 10, self.altura * 10))  # Esquerda
        pygame.draw.rect(tela, (200, 200, 200), ((self.largura - 1) * 10, 0, 10, self.altura * 10))  # Direita
        

        # Desenha os órgãos
        for celula in self.olhos + self.nariz + self.boca:
            pygame.draw.rect(tela, celula.get_cor(), (celula.get_x() * 10, celula.get_y() * 10, 10, 10))

        # Desenha os leucócitos
        for leucocito in self.leucocitos:
            pygame.draw.rect(tela, leucocito.get_cor(), (leucocito.get_x() * 10, leucocito.get_y() * 10, 10, 10))

        # Desenha os vírus influenza
        for virus in self.virus_influenza:
            pygame.draw.rect(tela, virus.get_cor(), (virus.get_x() * 10, virus.get_y() * 10, 10, 10))
        
        pygame.display.flip()  # Atualiza a tela

    def executarCiclo(self):
        self.moverCelulas()
        self.intera_Influenza()
        self.intera_Leucocito()
        self.up_Leucocitos()



In [26]:
def main():
    largura, altura = 60, 30
    pygame.init()
    tela = pygame.display.set_mode((largura * 10, altura * 10))
    pygame.display.set_caption("Simulação do Sistema Imunológico")
    clock = pygame.time.Clock()

    cabeca = Cabeca(largura, altura)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        cabeca.executarCiclo()
        cabeca.desenhar(tela)
        clock.tick(5)  # Limita a taxa de atualização para 5 quadros por segundo


In [27]:
if __name__ == "__main__":
    main()